In [1]:
!pip install transformers textblob vaderSentiment torch


In [2]:
# Step 1: Install Required Libraries
!pip install transformers textblob vaderSentiment torch

In [3]:


# Step 2: Import Libraries
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch

# Step 3: Load Translation Model
model_name = 'facebook/m2m100_418M'
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Step 4: Define Sentiment Analysis Functions
def analyze_sentiment(text):
    # Using TextBlob
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity

    # Using VADER
    analyzer = SentimentIntensityAnalyzer()
    vader_scores = analyzer.polarity_scores(text)

    return polarity, vader_scores

def categorize_emotion(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Step 5: Define Translation Function with Emotion Contextualization
def translate_with_emotion(text, target_lang):
    # Analyze sentiment
    polarity, vader_scores = analyze_sentiment(text)
    emotion = categorize_emotion(polarity)

    # Translate text
    tokenizer.src_lang = "en"  # Source language is English
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # Set the target language
    target_lang_id = tokenizer.get_lang_id(target_lang)
    model.config.forced_bos_token_id = target_lang_id

    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text, emotion, vader_scores

# Step 6: Test the Model with a Sample Text
sample_text = "I am so happy to see you!"
translated_text, emotion, vader_scores = translate_with_emotion(sample_text, "es")

print(f"Original Text: {sample_text}")
print(f"Translated Text: {translated_text}")
print(f"Emotion: {emotion}")
print(f"VADER Scores: {vader_scores}")

# Step 7: Load and Process Your Datasets
def process_dataset(file_path, target_lang):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    results = []
    for line in lines:
        translated_text, emotion, vader_scores = translate_with_emotion(line.strip(), target_lang)
        results.append((line.strip(), translated_text, emotion, vader_scores))
    
    return results

train_results = process_dataset('clean_train.txt', "es")
test_results = process_dataset('clean_test.txt', "es")
val_results = process_dataset('clean_val.txt', "es")

# Step 8: Evaluate the Model
def evaluate_model(results):
    correct_predictions = 0
    total = len(results)
    
    for original, translated, emotion, vader_scores in results:
        # Compare the predicted emotion with the original emotion
        predicted_emotion = categorize_emotion(analyze_sentiment(original)[0])
        if predicted_emotion == emotion:
            correct_predictions += 1

    accuracy = correct_predictions / total
    return accuracy

# Evaluate on validation data
val_accuracy = evaluate_model(val_results)
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Step 9: Re-evaluate on the Test Set
test_accuracy = evaluate_model(test_results)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Step 10: Save the Final Model and Results
with open('final_val_results.txt', 'w') as f:
    for original, translated, emotion, vader_scores in val_results:
        f.write(f"Original: {original}\n")
        f.write(f"Translated: {translated}\n")
        f.write(f"Emotion: {emotion}\n")
        f.write(f"VADER Scores: {vader_scores}\n")
        f.write("-" * 50 + "\n")

with open('final_test_results.txt', 'w') as f:
    for original, translated, emotion, vader_scores in test_results:
        f.write(f"Original: {original}\n")
        f.write(f"Translated: {translated}\n")
        f.write(f"Emotion: {emotion}\n")
        f.write(f"VADER Scores: {vader_scores}\n")
        f.write("-" * 50 + "\n")

# List of languages
languages = [
    {"name": "Hindi", "code": "hi"},
    {"name": "Tamil", "code": "ta"},
    {"name": "Spanish", "code": "es"},
    {"name": "French", "code": "fr"},
    {"name": "German", "code": "de"},
    {"name": "Chinese", "code": "zh"},
]

# List of Indian languages
indian_languages = ["hi", "ta"]

# Sample text to translate
sample_text = "I am so happy to see you!"

# Loop through Indian languages and print translations
for lang in languages:
    if lang["code"] in indian_languages:
        translated_text = translate_with_emotion(sample_text, lang["code"])
        print(f"Translated to {lang['name']}: {translated_text}")


/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Original Text: I am so happy to see you!
Translated Text: ¡Estoy muy contento de verlo!
Emotion: positive
VADER Scores: {'neg': 0.0, 'neu': 0.544, 'pos': 0.456, 'compound': 0.7213}


In [ ]:
import pickle
with open('model4.pickle','wb') as f:
    pickle.dump(model,f)